# Split background audio clips

The point of this notebook is to take sounds recorded on my terrace and split them into .5 to 1 second audio segments that I can use to train an audio classifier.

More specifically, this notebook does the following:
1. Splits the audio clips in `data/background/original` into segments of 1 second, saves them in `data/background/segmented`
2. Splits the audio clips in `data/atom/original` into non-silent segments of .5 to 1 second, saves them in `data/atom/segmented`
3. We pick a random sample from each of the folders and replay some of the clips


In [1]:
# imports and definitions
import librosa as lr
import librosa.display
import random
import warnings
import typing
import noisereduce as nr
import soundfile as sf
import numpy as np

from pathlib import Path
from IPython.display import Audio, IFrame, display
from collections import defaultdict

from src.sound import reduce_noise, split_audio_on_silence, split_audio_on_time, open_waveform, save_waveform

warnings.filterwarnings('ignore')


## Utility functions

In [2]:
def load_audio_and_display(input_file:Path):
    y, sr = librosa.load(input_file)
    display(Audio(y, rate = sr))

def display_segments(input_dir:Path, qty:int = 5, case:str = 'background', format = 'WAV'):
    sample_clips = random.choices(list(input_dir.glob(f"*.{format}".lower())), k = qty)
    for sample_clip in sample_clips:
        print(f"segmented background {case} clip ({sample_clip.stem})")
        load_audio_and_display(sample_clip)

def segment_audio(input_dir:Path, output_dir:Path, mode:str = 'regular', limits:list = [.5, 1.], format = 'WAV'):
    for input_filepath in list(input_dir.glob(f"*.{format}".lower())):
        # skip audio segmentation if output folder already contains segments of the original audio file
        if list(output_dir.glob(f"{input_filepath.stem}_*.{format}".lower())):
            continue

        # segment background audio clips
        waveform, sample_rate = open_waveform(input_filepath)
        if mode == 'regular':
            waveforms, sample_rate = split_audio_on_time(waveform, sample_rate, duration = max(limits), trim = False)
        elif mode == 'non-silent':
            waveforms, sample_rate = split_audio_on_silence(waveform, sample_rate, limits = limits, remove_noise = True)
        else:
            raise ValueError(f"did not recognize 'mode' value : {mode}")

        print(f"{input_filepath.stem} split into {len(waveforms)} waveforms")

        # save segments
        for i, waveform in enumerate(waveforms):
            filename = f"{input_filepath.stem}_{i}.{format}".lower()
            output_filepath = output_dir / filename
            save_waveform(waveform, sample_rate, output_filepath, format = format)


## Split background sound clips

In [3]:
# background clips directory
BACKGROUND_CLIPS_DIR = Path('../../data/background')
assert BACKGROUND_CLIPS_DIR.is_dir(), \
    f"directory {BACKGROUND_CLIPS_DIR} does not exist!"

# background/original clips directory
background_original_dir = BACKGROUND_CLIPS_DIR / 'original'
assert background_original_dir.is_dir(), \
    f"directory {background_original_dir} does not exist!"

# background/segmented clips directory
background_segmented_dir = BACKGROUND_CLIPS_DIR / 'segmented'
assert background_segmented_dir.is_dir(), \
    f"directory {background_segmented_dir} does not exist!"

segment_audio(background_original_dir, background_segmented_dir, mode = 'regular', limits = [.5, 1.], format = 'WAV')


2022_06_06_12_25_31 split into 15 waveforms
2022_06_10_15_32_01 split into 2171 waveforms
2022_06_10_16_32_42 split into 3341 waveforms
2022_06_05_13_38_27 split into 60 waveforms
2022_06_12_21_01_20 split into 1501 waveforms
2022_06_13_20_39_20 split into 2889 waveforms


## Pick and display segments from background sound clips


In [4]:
display_segments(background_segmented_dir, qty = 5, case = 'background', format = 'WAV')


segmented background background clip (2022_06_10_15_32_01_1742)


segmented background background clip (2022_06_10_15_32_01_596)


segmented background background clip (2022_06_10_15_32_01_437)


segmented background background clip (2022_06_12_21_01_20_594)


segmented background background clip (2022_06_13_20_39_20_2668)


## Split 'Atomic' sound clips

'Atom' is the name [of my dog](https://photos.app.goo.gl/pv34uvV49LqeM8vdA), a very rambunctious Vizsla.

When he whines - typically when he's asking me to play with him or simply asking for food - the noise he makes is similar to that of a Yellow-legged Gull (at least to my ears...), so I have to make sure the classifier is able to distinguish between the two.


In [5]:
# atom clips directory
ATOM_CLIPS_DIR = Path('../../data/atom')
assert ATOM_CLIPS_DIR.is_dir(), \
    f"directory {ATOM_CLIPS_DIR} does not exist!"

# atom/original clips directory
atom_original_dir = ATOM_CLIPS_DIR / 'original'
assert atom_original_dir.is_dir(), \
    f"directory {atom_original_dir} does not exist!"

# atom/segmented clips directory
atom_segmented_dir = ATOM_CLIPS_DIR / 'segmented'
assert atom_segmented_dir.is_dir(), \
    f"directory {atom_segmented_dir} does not exist!"

segment_audio(atom_original_dir, atom_segmented_dir, mode = 'non-silent', limits = [.5, 1.], format = 'WAV')


2022_06_12_21_01_21 split into 13 waveforms
2022_06_12_21_01_20 split into 12 waveforms
2022_06_15_11_52_37 split into 36 waveforms


## Pick and display segments from 'Atomic' sound clips

... ok, there may be some clips with human voices in there too


In [6]:
display_segments(atom_segmented_dir, qty = 5, case = 'Atom', format = 'WAV')


segmented background Atom clip (2022_06_12_21_01_20_4)


segmented background Atom clip (2022_06_12_21_01_21_7)


segmented background Atom clip (2022_06_15_11_52_37_14)


segmented background Atom clip (2022_06_15_11_52_37_8)


segmented background Atom clip (2022_06_15_11_52_37_2)
